# Assignment 5

## Step 1
Modify your program that calculates beta to take into the beta() method a frequency (day, month) and a duration ( number of years) to modify the beta calculation.

## Step 2
Create a historical diagram of the S&P index, Bitcoin, Ethereum, and XRP daily prices.

## Step 3
Write your datasets and the graph image from your Python program into a Microsoft Excel spreadsheet.

## Step 4
Implement a version of your program as a Python program instead of Jupyter Notebook.

## Converting a Jupyter IPython Notebook to a simple Python program file
For those looking for simple ways to convert your Jupyter  notebook to a stand alone Python program here is a good reference:

Convert Python notebook to Python (Medium) by Vaku Lois (Links to an external site.) 
https://medium.com/@lois_idzi/convert-json-ipython-notebook-ipynb-to-python-py-file-e8cb42ba1455
 

## Link to assignment
 
https://mtu.instructure.com/courses/1347059/assignments/9202110

 ## Step 1
 
 The approach to calculate beta was taken from:
 https://www.codingfinance.com/post/2018-04-25-portfolio-beta-py/

 (Note: The example on codingfinance.com is rife with errors)
 
 Idea is myanalysis = StockAnalyzer(listOfStocks)
 
 Alternative ways to invoke - specify datasource
 
 myanalysis2 = StockAnalyzer(listOfStocks, listofweights, datasource="yahoo")
 
 Specify a date in the past to calculate a previous beta
 
 myanalysis2=  StockAnalyser(listOfStocks, listofweights, asofDate='2013-03-31')

In [3]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn
import pandas_datareader as pdr
import pandas as pd
import datetime
import yfinance as yf
from dateutil.relativedelta import relativedelta
#
# Packages needed to write to Excel
#
import openpyxl
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Fill, PatternFill,Border
#
# StockAnalyzer class that will calculate betas for a portfolio of stocks
#

class StockAnalyzer():

    def __init__(self,listOfStocks=[], listOfWeights=[], datasource="yahoo", asofDate="", betaperiod=5, market="SPY"):

        self._stocks=listOfStocks
        self._portfolioWeights=listOfWeights
        self.__datasource=datasource
        self.__betaperiod=betaperiod
        self._market=market

        if asofDate=="":
            self._asofDate=datetime.date.today()
        else:
            self._asofDate=datetime.strpTime(asofDate)

        self._startDate=self._asofDate + relativedelta(years=-self.__betaperiod)

        self._stockprices = pdr.DataReader(self._stocks, data_source = self.__datasource,
                                           start = self._startDate, end = self._asofDate)['Adj Close']

        self._marketprice = pdr.DataReader(self._market, data_source = self.__datasource,
                                           start = self._startDate, end = self._asofDate)['Adj Close']

        
    def stockPrices(self):
        return self._stockprices

    def setMarketTicker(self,newmarket):
        self._market=newmarket

    def portfoliobetaDaily(self):
        return beta(self._marketprice,self._stockprices, self._portfolioWeights)
       
        
    def portfoliobetaMonthly(self):
        return beta(self._marketprice.resample('M').last(),self._stockprices.resample('M').last(),self._portfolioWeights)
 
#
# Stand alone function (not part of the StockAnalyzer class) that takes in a list of stocks and then queries Yahoo! to read a specific attribute
# Usage example is  stockinfo(["TSLA","GM"],"beta")
#
# returns a pandas dataframe contain the stocks and the attribute specified
#       
def stockinfo(stocks, attribute):
    
    stockattribute=pd.DataFrame()

    for stock in stocks:
        stockinfo=yf.Ticker(stock).info
        stockattribute=stockattribute.append({"Stock":stock,attribute:stockinfo[attribute]},ignore_index=True)

    return stockattribute

#
# Stand alone function (not part of the StockAnalyzer class) to calculate the beta for a portfolio of stocks in addition to the betas for each stock and a list of the weights for 
# each stock in the portfolio
#
# expects a pandas dataframe of prices for a benchmark security and a pandas dataframe containing prices for a list of stocks
#
# return a pandata dataframe with the betas for the portfolio 
#
#
def beta(benchmark,portfolio,portfolioWeights):

    # calculate the returns (percent change)
    
    benchmark_returns = benchmark.pct_change()[1:]

    stock_returns = portfolio.pct_change()[1:]


    # calculate portfolio beta and store in a new dataframe

    beta=pd.DataFrame()

    # calculate the beta for each stock

    for s in stock_returns.columns:

        result = sp.stats.linregress(benchmark_returns.values,stock_returns[s].values)

        beta=beta.append({"Stock":s,"beta":result.slope},ignore_index=True)       
 
    # create the portfolio returns by taking the weight of each return
    
    portfolio_returns = (stock_returns * portfolioWeights).sum(axis = 1)
    
    result = sp.stats.linregress(benchmark_returns.values,portfolio_returns.values)

    beta=beta.append({"Stock": "Portfolio","beta":result.slope},ignore_index=True)    
 
    
    return beta

## Demonstrate the use of mystockanalyzer

In [4]:
mystocks=["TSLA","GME","T","IBM","AAPL"]

myweights=[0.2,0.2,0.2,0.2,0.2]

mystockanalyzer = StockAnalyzer(mystocks,myweights)

yahoobetas=stockinfo(mystocks,"beta")

dailybetas=mystockanalyzer.portfoliobetaDaily()

monthlybetas=mystockanalyzer.portfoliobetaMonthly()

print("returned from Yahoo! Finance")
print(yahoobetas)

print("Calculated using daily prices")
print(dailybetas)

print("calculated using monthly prices")
print(monthlybetas)

returned from Yahoo! Finance
  Stock      beta
0  TSLA  2.010479
1   GME -1.816878
2     T  0.737451
3   IBM  1.225041
4  AAPL  1.219525
Calculated using daily prices
       Stock      beta
0       TSLA  1.339685
1        GME  0.792031
2          T  0.756354
3        IBM  0.991783
4       AAPL  1.210407
5  Portfolio  1.018052
calculated using monthly prices
       Stock      beta
0       TSLA  1.987548
1        GME -2.116980
2          T  0.776212
3        IBM  1.250217
4       AAPL  1.218496
5  Portfolio  0.623099


## Extras: Outputting your betas to Excel
This is not part of the assignment - but might be useful for Step 2, which is to get pricing data and output to an Excel spreadsheet


### Approach 1: simply output the pandas DataFrame to individual files


In [5]:
yahoobetas.to_excel("yahoobetas.xlsx")
dailybetas.to_excel("dailybetas.xlsx")
monthlybetas.to_excel("monthlybetas.xlsx")

### Approach 2: Use the openpyxl package to output all the betas to different sheets in the same Excel

In [6]:
#
# Write them to different sheets in the same workbook
#
with pd.ExcelWriter("mybetas.xlsx") as writer:
    yahoobetas.to_excel(writer,sheet_name='Yahoo_beta')
    dailybetas.to_excel(writer,sheet_name='Daily_beta')
    monthlybetas.to_excel(writer,sheet_name='Monthly_beta')

### Approach 3: Write all the data in a single sheet using openpyxl

In [7]:
# create a workbook object in memory 

wb=Workbook() # Empty workbook with one default sheet

# ask the workbook object for its active sheet (the one default one added)

ws=wb.active

# Change the tab title of the worksheet

ws.title="Betas"

# Let's access the propertis of or worksheet so we can give our worksheet tab a color

wsprops = ws.sheet_properties
wsprops.tabColor = "00FF00"

# add a header row

columns=['Ticker', 'Yahoo!','Daily','Monthly']

ws.append(columns)

# Lets color the header row cells and bold the text by setting the fill and font attributes on each cell


fill1=PatternFill(fill_type="solid",start_color="FFFF00", end_color="FFFF00")

# We can use the length of the columns list from above to determine how many cells to make bold and turn yellow.

for i in range(1,len(columns)+1):
    ws.cell(row=1,column=i).fill=fill1
    ws.cell(row=1,column=i).font=Font(bold=True)

#
# Approach 1: we will convert Yahoo Betas to lists and then extend that list with values from the daily and monthly
# betas and add them to the Yahoo betas list, then append the entire row to our spreadsheet.
#

for i in range(yahoobetas.index.size):
    row=list(yahoobetas.iloc[i,0:2])
    row.extend(list(dailybetas.iloc[i,1:2])) # add the value from daily to the list
    row.extend(list(monthlybetas.iloc[i,1:2])) # add the value from monthly to the list
    print(row)
    ws.append(row)

wb.save("combined.xlsx")

['TSLA', 2.010479, 1.3396847088812052, 1.9875478922370142]
['GME', -1.816878, 0.7920311078621645, -2.1169804848017924]
['T', 0.737451, 0.7563539896367435, 0.7762121386012019]
['IBM', 1.225041, 0.9917834379005854, 1.2502167787404872]
['AAPL', 1.219525, 1.2104070743144126, 1.2184964885935048]



### Approach 4: we will use the Pandas Dataframe merge capability to create on dataframe then export it to Excel.


In [8]:
# We will merge daily and monthly betas - the duplicate column names will be appended with _daily and _monthly respectively

combined=pd.merge(dailybetas, monthlybetas, left_on = 'Stock', right_on = 'Stock', how = 'left', suffixes=("_daily","_monthly"))

# Now merge the combined with the yahoo betas

combined=pd.merge(yahoobetas,combined, left_on = 'Stock', right_on = 'Stock', how = 'left')

# since all the data is in a single dataframe now - we can just ask pandas to output as an Excel spreadsheet

combined.to_excel("combined_simple.xlsx")

print(combined)

  Stock      beta  beta_daily  beta_monthly
0  TSLA  2.010479    1.339685      1.987548
1   GME -1.816878    0.792031     -2.116980
2     T  0.737451    0.756354      0.776212
3   IBM  1.225041    0.991783      1.250217
4  AAPL  1.219525    1.210407      1.218496
